In [2]:
import os
import numpy as np
import pandas as pd
import json
import geopandas as gpd
from pprint import pprint
from copy import deepcopy
from geopy.distance import geodesic as GD
from shutil import move

levels = ['province','district','commune']

this_path = os.getcwd()
print(this_path)

c:\Users\veasn\OneDrive - TUX Global Institute\AI001 Module 2 - Assignment 3


# Clean Communes

In [ ]:

df1 = pd.read_json('Map/data_with_neighbors.json',dtype={"Geocode":str})
dft1 = pd.read_csv('Map/data.csv',dtype={"Geocode":str})
print('Shape1:',df1.shape)
print('Shape2:',dft1.shape)
csvdata = []
for i, row in df1.iterrows():
    neighbors = row['neighbors']
    for neighbor in neighbors:
        csvdata.append({
            'Order': row['Order'],
            'Geocode': row['Geocode'],
            'Name': row['Name'],
            'Neighbor Name': neighbor,
            # 'Neighbor Code': '',
        })
    # if len(neighbors) == 0:
    #     print("%5d  %6s  %s" % (row['Order'],row['Geocode'],row['Name']))
df3 = pd.DataFrame(csvdata)
print('Shape3:',df3.shape)
print('Geocode:',df3['Geocode'].nunique())
print('Missing:',df1['Geocode'].nunique() - df3['Geocode'].nunique())


gp = dft1.groupby('Name',as_index=False)\
    .agg({'Geocode':'first','Order':'count'})\
    .rename(columns={'Order':'Count'})\
    .sort_values('Count',ascending=False)\
    .query('Count==1')
print('Group :',gp.shape)
print('Name  :',gp['Name'].nunique())
gp.to_csv('test.csv',index=False)
gp = gp[['Geocode','Name']].rename(columns={'Geocode':'Neighbor Code','Name':'Neighbor Name'})
df3 = pd.merge(df3, gp, how='left', on='Neighbor Name')

# for i,row in df3.iterrows():
#     print(i)
#     for _, row2 in gp.iterrows():
#         if row['Neighbor Name'] == row2['Name']:
#             df3.at[i,'Neighbor Code'] = row2['Geocode']
#             break

print('Shape3:',df3.shape)
print('Null  :',df3['Neighbor Code'].isnull().sum())
df3.to_csv('Map/communes_neighbors.csv',index=False)
df3[df3['Neighbor Code'].isna()].to_csv('test_null.csv',index=False)


# Find Neighbors

In [59]:

gdf = gpd.read_file('Map/communes.json', encoding='utf-8')
print('\n# communes.json')
print('shape   :',gdf.shape)
print('columns :',gdf.columns.to_list())
print('hrpcode :',gdf['hrpcode'].nunique())
print('hrparent:',gdf['hrparent'].nunique())
# print(gdf.loc[0,:])
# print(gdf.info())

print('\n# data.json')
data = json.load(open('Map/data.json', mode='r', encoding='utf-8'))
print('rows:',len(data))
missing_communes = []
for i,row in enumerate(data):
    commune_name = row['Name']
    commune_code = row['Geocode']
    target_commune = gdf[gdf['com_code'] == commune_code]
    data[i]['Neighbors'] = []
    if target_commune.empty:
        missing_communes.append({
            'Geocode': row['Geocode'],
            'Name': row['Name'],
        })
    else:
        neighbors = gdf[gdf.geometry.touches(target_commune.geometry.iloc[0])]
        data[i]['Neighbors'] = neighbors['com_code'].to_list()
print('Not Found:',len(missing_communes))
with open('Map/test_neighbor.json', 'w', encoding='utf-8') as fw:
    json.dump(data, fw, ensure_ascii=False, indent=4)
pd.DataFrame(missing_communes).to_csv('Map/test_missing.csv',index=False)



# communes.json
shape   : (1633, 11)
columns : ['id', 'objectid', 'level', 'hrname', 'hrpcode', 'hrparent', 'com_name', 'com_code', 'dis_code', 'pro_code', 'geometry']
hrpcode : 1633
hrparent: 197

# data.json
rows: 1652
Not Found: 98


# Gazetteer

In [8]:

fileR = 'Gazetteer/gazetteer.csv'
df1 = pd.read_csv(fileR)
dft1 = pd.read_csv('Map/data.csv',dtype={"Geocode":str})
df3 = pd.read_json('Map/data.json',dtype={"Geocode":str})
df1 = df1.query('Level==3')
dft1['Code'] = dft1['Geocode'].apply(lambda x: f'KH{x}' if len(x) % 2 == 0 else f'KH0{x}')
print('Shape1:',df1.shape)
print('Shape2:',dft1.shape)
print('Shape3:',df3.shape)
print('Geocode2:',dft1['Geocode'].nunique())
print('Geocode3:',df3['Geocode'].nunique())
# set1 = set(df1['Code'].unique())
# set2 = set(df2['Code'].unique())
# print('set1:',set1.difference(set2))
# print('set2:',set2.difference(set1))
# df2 = pd.merge(df2, df3[['Geocode','Latitude','Longitude']], how='left', on='Geocode')
# del df2['Code']
# df2.to_csv('data.csv',index=False)
# gp = df2.groupby('Name',as_index=False)\
#     .agg({'Order':'count'})\
#     .rename(columns={'Order':'Count'})\
#     .sort_values('Count',ascending=False)\
#     .query('Count>1')
# print(gp.shape)
# gp.to_csv('test.csv',index=False)


Shape1: (1652, 8)
Shape2: (1652, 10)
Shape3: (1652, 9)
Geocode2: 1652
Geocode3: 1652


In [29]:

df = pd.read_csv('data/Map/geonode.csv').query('level==3')
columns = df.columns
print('Shape:',df.shape)
data = json.load(open('data/Map/graph.json', mode='r', encoding='utf-8'))
print('Json :',len(data))


fileR = 'data/Map/geonode/commune.json'
fileW = 'data/Map/test.json'
with open(fileR, mode='r', encoding='utf-8') as fr,\
    open(fileW, mode='w', encoding='utf-8') as fw:
    data = json.load(fr)
    print('Json :',len(data['features']))
    print(data['features'][0]['properties'].keys())
    properties = []
    for i,row in enumerate(data['features']):
        source_adm = row['properties']['source_adm']
        geocode = f'KH{source_adm}' if len(source_adm) % 2 == 0 else f'KH0{source_adm}'
        dfx = df[df['code'] == geocode]
        if dfx.shape[0] == 1:
            data['features'][i]['properties']['adm3_altnm'] = dfx.iloc[0]['name_km']
        else:
            print('Not Found:', source_adm, geocode, row['properties']['adm3_altnm'])
        properties.append(row['properties'])
    json.dump(data, fw, separators=(',', ':')) 
    # json.dump(properties, fw, indent=4)


# for i,row in enumerate(data):
#     dfx = df[df['code'] == row['code']]
#     if dfx.shape[0] == 1:
#         data[i]['name_km'] = dfx.iloc[0]['name_km']
#     else:
#         print('Not Found',i)

# with open('data/Map/test.json', mode='w', encoding='utf-8') as fw:
#     json.dump(data, fw, indent=4) 

Shape: (1632, 8)
Json : 1632
Shape2: (4710, 8)
codes : 9420

# Results
Shape X1: (1632, 2)
Shape X2: (1632, 2)
Shape X3: (1632, 12)
equal
1    1500
0     132
Name: count, dtype: int64


In [3]:

df = pd.read_csv('data/Map/geonode.csv').query('level==3')
columns = df.columns
print('Shape:',df.shape)
data = json.load(open('data/Map/graph.json', mode='r', encoding='utf-8'))
print('Json :',len(data))

for i,row in enumerate(data):
    data[i]['count'] = len(row['neighbors'])
    data[i]['neighbors'] = set([x['code'] for x in row['neighbors']])
# pprint(data[0])

df2 = pd.read_csv('data\Map\geonode_neighbor.csv').query('level==3')
codes1 = df2['code1'].to_list()
codes2 = df2['code2'].to_list()
codes = codes1 + codes2
print('Shape2:',df2.shape)
print('codes :',len(codes))

def get_neighbors(code):
    neighbor_set = {code}
    dfx = df2[(df2['code1']==code) | (df2['code2']==code)]
    set1 = set(dfx['code1'].to_list())
    set2 = set(dfx['code2'].to_list())
    neighbor_set = neighbor_set.union(set1)
    neighbor_set = neighbor_set.union(set2)
    neighbor_set.remove(code) 
    return neighbor_set

df['neighbors'] = df['code'].apply(get_neighbors)
# df['neighbors'] = df['code'].apply(lambda x: codes.count(x))
# for i,row in df.iterrows():



print('\n# Results')
dft1 = pd.DataFrame(data,columns=['code','neighbors'])
dft2 = df[['code','neighbors']].copy()
dft1['num'] = dft1['neighbors'].apply(lambda x: len(x))
dft2['num'] = dft2['neighbors'].apply(lambda x: len(x))
dft3 = pd.merge(dft1, dft2, how='left', on='code',suffixes=('_M','_V'))
dft3['equal'] = np.where(dft3['neighbors_M']==dft3['neighbors_V'], 1, 0)
dft3 = pd.merge(dft3, df, how='left', on='code')
print('Shape X1:',dft1.shape)
print('Shape X2:',dft2.shape)
print('Shape X3:',dft3.shape)
print(dft3['equal'].value_counts())
dft1.sort_values('code').to_csv('test_1.csv',index=False)
dft2.sort_values('code').to_csv('test_2.csv',index=False)
dft3.query('equal==0').sort_values('code').to_csv('test_3.csv',index=False,columns="code,num_M,num_V,name_en,name_km,neighbors_M,neighbors_V".split(','))


Shape: (1632, 8)
Json : 1632
Shape2: (4710, 8)
codes : 9420

# Results
Shape X1: (1632, 3)
Shape X2: (1632, 3)
Shape X3: (1632, 14)
equal
1    1495
0     137
Name: count, dtype: int64


In [5]:

json_data = json.load(open('data/Map/template.json', mode='r', encoding='utf-8'))
data = json.load(open('data/Map/graph.json', mode='r', encoding='utf-8'))
print()
print(len(data))
temp = json_data["features"][0].copy()
json_data["features"].clear()
csvdata = []
for row in data:
    # if row['code'] == 'KH010201':
    #     for neighbor in row['neighbors']:
    #         print(neighbor)
    #         obj = temp.copy()
    #         obj['properties']['code'] = neighbor['code']
    #         obj['properties']['name'] = neighbor['name']
    #         obj['geometry']['coordinates'] = [neighbor['lng'],neighbor['lat']]
    #         json_data["features"].append(deepcopy(obj))
    #     break

    csvdata.append({
        "order": row["Order"],
        "code": row["code"],
        "name_en": row["name_en"],
        "name_km": row["name_km"],
        "longitude": row["Longitude"],
        "latitude": row["Latitude"],
    })


df = pd.DataFrame(csvdata)
print('Shape:',df.shape)
print('code :',df['code'].nunique())
df.to_csv('test.csv',index=False)

# with open('data/Map/test.json', mode='w', encoding='utf-8') as fw:
#     json.dump(json_data, fw, indent=4) 


2
1632
Shape: (1632, 6)
code : 1632


In [ ]:

from geopy.distance import geodesic as GD
point1 = (row['latitude'] , row['longitude'])
point2 = (row['latitude'] , row['longitude'])
distance = GD(point1,point2).km


In [2]:
filepath = os.path.join(this_path, 'data/Map/graph.json')

with open(filepath, mode='r', encoding='utf-8') as fr,\
    open('test.json', mode='w', encoding='utf-8') as fw:
    data = json.load(fr)
    node_lookup = {node['code']: node for node in data}
    g_scores = {node['code']: float('inf') for node in data}
    json.dump(g_scores, fw, indent=4) 


In [81]:


df = pd.read_csv('data/Map/geonode.csv').query('level==3')
columns = df.columns
print('Shape:',df.shape)
data = json.load(open('static\json\commune.json', mode='r', encoding='utf-8'))
print('Json :',len(data["features"]))


Shape: (1632, 8)
Json : 1632


In [3]:

os.remove('test.json')